In [1]:
from sklearn.datasets import load_iris
from sklearn import tree
import pandas as pd
import numpy as np
from ddal import DDAL
from batcher import BatchGenerator

iris = load_iris()

In [2]:
df_x = pd.DataFrame(data= np.c_[iris['data'], iris['target']], columns= iris['feature_names'] + ['target']).sample(frac=1, random_state=42)
df_y = df_x.pop('target')

df_x_train = df_x[0:50]
df_y_train = df_y[0:50]


In [3]:
classifier = tree.DecisionTreeClassifier()
classifier.fit(df_x_train, df_y_train)

DecisionTreeClassifier()

In [4]:
ddal = DDAL(size_batch=50, theta=0.005, lambida=0.95)

df_x_test_batch_1 = df_x[50:100]
df_y_test_batch_1 = df_y[50:100].to_frame()

df_x_test_batch_1.reset_index(inplace=True, drop=True)
df_y_test_batch_1.reset_index(inplace=True, drop=True)

In [5]:
for index, row in df_x_test_batch_1.iterrows():
    y_pred = classifier.predict_proba(df_x_test_batch_1.iloc[[index]])
    max_y_pred_prob = y_pred.max()
    ddal.count_selected_instances(max_y_pred_prob)

ddal.compute_current_density()

In [6]:
if ddal.detection_module():
    print('Drift Detected')
    ddal.reset()

### test if it will detect that data comes from different distribution

In [7]:
data_sin = pd.read_csv('../data/Sine.csv', index_col=0)
data_sin['Classification'] = data_sin['Classification'].map({'Initially Negative':0,'Initially Positive':1})
data_sin['Concept Drift'] = data_sin['Concept Drift'].map({'After Concept Drift Negative':0,'After Concept Drift Positive':1})
data_sin.head()

,x,y,Classification,Concept Drift
0,0.048725,0.275135,0,1
1,0.289110,0.186188,1,0
2,0.720966,0.077742,1,0
3,0.021616,0.916041,0,1
4,0.205923,0.041295,1,0


In [8]:
data_sin_tmp = data_sin.copy()
df_y = data_sin_tmp['Classification']

two_thirds = int(2*len(data_sin)/3)
df_x_train = data_sin_tmp[0:two_thirds].drop(columns=['Concept Drift','Classification'])
df_y_train = df_y[0:two_thirds]

data_test = data_sin_tmp.iloc[two_thirds:len(data_sin_tmp)].reset_index(drop=True)

In [9]:
classifier = tree.DecisionTreeClassifier()
classifier.fit(df_x_train, df_y_train)

DecisionTreeClassifier()

In [17]:
batcher = BatchGenerator(50,data_test)
ddal = DDAL(size_batch=50, theta=0.000005, lambida=0.90)

In [18]:
for batch_test in batcher:
    batch_test = batch_test.drop(columns=['Concept Drift','Classification'])
    for index,_ in batch_test.iterrows():
        y_pred = classifier.predict_proba(batch_test.iloc[[index]])
        max_y_pred_prob = y_pred.max()
        ddal.count_selected_instances(max_y_pred_prob)

    ddal.compute_current_density()
    
    if ddal.detection_module():
        print('Drift Detected')
        ddal.reset()

All data has been fetched.
